### Preprocessing and some visualizations

In [589]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


In [590]:
url = 'https://storage.googleapis.com/esmartdata-courses-files/ml-course/OnlineRetail.csv'
raw_data = pd.read_csv(url, encoding='latin', parse_dates=['InvoiceDate'])
df = raw_data.copy()
df.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
160805,550470,22497,SET OF 2 TINS VINTAGE BATHROOM,1,2011-04-18 13:49:00,8.29,NaN,United Kingdom
325441,565441,23394,POSTE FRANCE CUSHION COVER,1,2011-09-04 14:08:00,3.75,13240.0,United Kingdom
324331,565396,85131D,BEADED CRYSTAL HEART PINK ON STICK,1,2011-09-02 16:39:00,0.83,NaN,United Kingdom
192622,553462,23160,REGENCY TEA SPOON,12,2011-05-17 10:57:00,1.25,17147.0,United Kingdom
18281,537818,22867,HAND WARMER BIRD DESIGN,12,2010-12-08 13:53:00,2.10,12766.0,Portugal
456459,575692,23211,RED ROCKING HORSE HAND PAINTED,2,2011-11-10 16:27:00,1.25,12508.0,France
273820,560862,79030D,TUMBLER BAROQUE,18,2011-07-21 13:48:00,0.39,14085.0,United Kingdom
125623,547052,21928,JUMBO BAG SCANDINAVIAN BLUE PAISLEY,2,2011-03-20 12:07:00,1.95,18225.0,United Kingdom
439780,574481,16218,CARTOON PENCIL SHARPENERS,5,2011-11-04 12:45:00,0.06,18022.0,United Kingdom
213763,555556,84970L,SINGLE HEART ZINC T-LIGHT HOLDER,12,2011-06-05 14:04:00,0.95,15704.0,United Kingdom


In [591]:
print('Percentage of null cells:')
df.isnull().sum() / len(df) 

Percentage of null cells:


InvoiceNo      0.000000
StockCode      0.000000
Description    0.002683
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
CustomerID     0.249267
Country        0.000000
dtype: float64

In [592]:
print('Before clenaing - ', len(df))
df.dropna(inplace=True)
print('Ater clenaing - ',  len(df))

Before clenaing -  541909
Ater clenaing -  406829


In [593]:
# frequency countries
country_counts = df['Country'].value_counts() / len(df['Country']) 

# percentage of frequency countries above 0.5%
main = country_counts[country_counts > 0.005].round(2)
rest = country_counts[country_counts < 0.005].sum().round(2)
main['Other Countries'] = rest        


fig = px.pie(values=main, names=main.index, color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
    legend_x=-0.5,
    width=800,
    height=600

)

In [594]:
# Consider only United Kingdom

df_uk = df[df.Country=='United Kingdom']

In [595]:
df_uk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361878 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    361878 non-null  object        
 1   StockCode    361878 non-null  object        
 2   Description  361878 non-null  object        
 3   Quantity     361878 non-null  int64         
 4   InvoiceDate  361878 non-null  datetime64[ns]
 5   UnitPrice    361878 non-null  float64       
 6   CustomerID   361878 non-null  float64       
 7   Country      361878 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.8+ MB


In [596]:
df_uk.describe()

,Quantity,UnitPrice,CustomerID
count,361878.000000,361878.000000,361878.000000
mean,11.077029,3.256007,15547.871368
std,263.129266,70.654731,1594.402590
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,14194.000000
50%,4.000000,1.950000,15514.000000
75%,12.000000,3.750000,16931.000000
max,80995.000000,38970.000000,18287.000000


In [597]:
df_uk.describe(include=['object'])

,InvoiceNo,StockCode,Description,Country
count,361878,361878,361878,361878
unique,19857,3661,3860,1
top,576339,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,542,1987,1980,361878


In [598]:
# Create new column - Sales

df_uk['Sales'] = df_uk['Quantity'] * df_uk['UnitPrice']
df_uk.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [599]:
# Summary and visualization sales group by date

df_uk_group = df_uk.groupby(df_uk['InvoiceDate'].dt.date)['Sales'].sum().reset_index()
df_uk_group.columns = ['InvoiceDate', 'Sales']
df_uk_group.head()


fig = px.line(df_uk_group, x='InvoiceDate', y='Sales', color_discrete_sequence=['#03fcb5'], width=700, height=300)
fig.show()

### Classification by summary sales

In [600]:
# Extract data and standardization 

from sklearn.preprocessing import StandardScaler

df_sales = df_uk.groupby('CustomerID')['Sales'].sum().reset_index()
df_sales.head()

scaler = StandardScaler()
df_sales['ScaledSales'] = scaler.fit_transform(df_sales[['Sales']])
df_sales.head()

,CustomerID,Sales,ScaledSales
0,12346.0,0.00,-0.261674
1,12747.0,4196.01,0.379155
2,12748.0,29072.10,4.178320
3,12749.0,3868.20,0.329091
4,12820.0,942.34,-0.117757


In [601]:
df_sales['ScaledSales']

0      -0.261674
1       0.379155
2       4.178320
3       0.329091
4      -0.117757
          ...   
3945   -0.234092
3946   -0.249331
3947   -0.234703
3948    0.058263
3949    0.018922
Name: ScaledSales, Length: 3950, dtype: float64

In [602]:
df_sales[['ScaledSales']]

,ScaledSales
0,-0.261674
1,0.379155
2,4.178320
3,0.329091
4,-0.117757
...,...
3945,-0.234092
3946,-0.249331
3947,-0.234703
3948,0.058263


In [603]:
# Clustering by DBSCAN

from sklearn.cluster import DBSCAN 

cluster = DBSCAN(eps=0.7, min_samples=4)
cluster.fit(df_sales[['ScaledSales']])

df_sales['ClusterDbscan'] = cluster.labels_
df_sales.head()

df_sales['ClusterDbscan'] = df_sales['ClusterDbscan'].map({1:2, -1:1, 0:0})
df_sales

,CustomerID,Sales,ScaledSales,ClusterDbscan
0,12346.0,0.00,-0.261674,0
1,12747.0,4196.01,0.379155,0
2,12748.0,29072.10,4.178320,0
3,12749.0,3868.20,0.329091,0
4,12820.0,942.34,-0.117757,0
...,...,...,...,...
3945,18280.0,180.60,-0.234092,0
3946,18281.0,80.82,-0.249331,0
3947,18282.0,176.60,-0.234703,0
3948,18283.0,2094.88,0.058263,0


In [604]:
# Clustering by K-means

from sklearn.cluster import KMeans

# elbow method
WCSS=[]
for n in range(1,10):
  cluster = KMeans(n_clusters=n)
  cluster.fit(df_sales[['ScaledSales']])
  WCSS.append(cluster.inertia_)
  
WCSS

fig = px.line(x=range(1,10), y=WCSS, width=500, height=300, template='xgridoff')
fig.show()

In [605]:
cluster = KMeans(n_clusters=3)
cluster.fit(df_sales[['ScaledSales']])
cluster.labels_

df_sales['ClusterKmeans'] = cluster.labels_
df_sales.head()


,CustomerID,Sales,ScaledSales,ClusterDbscan,ClusterKmeans
0,12346.0,0.00,-0.261674,0,0
1,12747.0,4196.01,0.379155,0,0
2,12748.0,29072.10,4.178320,0,2
3,12749.0,3868.20,0.329091,0,0
4,12820.0,942.34,-0.117757,0,0


In [606]:
# Clustering by Agglomerative method

from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=3)
cluster.fit(df_sales[['ScaledSales']])

df_sales['ClusterAgglomerative'] = cluster.labels_
df_sales.head()

,CustomerID,Sales,ScaledSales,ClusterDbscan,ClusterKmeans,ClusterAgglomerative
0,12346.0,0.00,-0.261674,0,0,0
1,12747.0,4196.01,0.379155,0,0,0
2,12748.0,29072.10,4.178320,0,2,2
3,12749.0,3868.20,0.329091,0,0,0
4,12820.0,942.34,-0.117757,0,0,0


In [607]:
df_sales['ClusterDbscan'] = df_sales['ClusterDbscan'].astype('object')
df_sales['ClusterKmeans'] = df_sales['ClusterKmeans'].astype('object')
df_sales['ClusterAgglomerative'] = df_sales['ClusterAgglomerative'].astype('object')
df_sales.dtypes

CustomerID              float64
Sales                   float64
ScaledSales             float64
ClusterDbscan            object
ClusterKmeans            object
ClusterAgglomerative     object
dtype: object

In [608]:

fig = make_subplots(rows=3 , cols=1, shared_yaxes='all', 
                    column_width=[3], subplot_titles=("DBSCAN", "KMeans", "Agglomerative"))


fig.add_trace(go.Scatter(x=df_sales['CustomerID'], y= df_sales["ScaledSales"], mode='markers', marker_color=df_sales["ClusterDbscan"], showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=df_sales[df_sales["ClusterDbscan"]==1]['CustomerID'], 
                           y= df_sales[df_sales["ClusterDbscan"]==1]["ScaledSales"],
                           mode='markers', marker_symbol='x', marker_color="lightskyblue", marker_size=10, showlegend=True), row=1, col=1)

fig.add_trace(go.Scatter(x=df_sales['CustomerID'], y= df_sales["ScaledSales"], mode='markers', marker_color=df_sales["ClusterKmeans"], showlegend=True), row=2, col=1)
fig.add_trace(go.Scatter(x=df_sales['CustomerID'], y= df_sales["ScaledSales"], mode='markers', marker_color=df_sales["ClusterAgglomerative"], showlegend=True), row=3, col=1)

fig.update_layout(showlegend=False, template='plotly_dark', width=1000, height=900,)

fig.show()

### Classification by total sales and last purchase

In [609]:
# Preparing DataFrame

df_purchase = df_sales.copy()[['CustomerID', 'Sales', 'ScaledSales']]
df_purchase['LastPurchase'] = df_uk.groupby('CustomerID')['InvoiceDate'].max().values

  # days from last purchase
last_date = df_purchase['LastPurchase'].max()
df_purchase['LastPurchase_days'] = last_date - df_purchase['LastPurchase']
df_purchase['LastPurchase_days'] = df_purchase['LastPurchase_days'].apply(lambda x:x.days)

scaler = StandardScaler()
df_purchase['ScaledPurchase'] = scaler.fit_transform(df_purchase[['LastPurchase_days']])

df_purchase.head()

,CustomerID,Sales,ScaledSales,LastPurchase,LastPurchase_days,ScaledPurchase
0,12346.0,0.00,-0.261674,2011-01-18 10:17:00,325,2.337128
1,12747.0,4196.01,0.379155,2011-12-07 14:34:00,1,-0.895835
2,12748.0,29072.10,4.178320,2011-12-09 12:20:00,0,-0.905813
3,12749.0,3868.20,0.329091,2011-12-06 09:56:00,3,-0.875878
4,12820.0,942.34,-0.117757,2011-12-06 15:12:00,2,-0.885857


In [610]:
# Clustering by DBSCAN

from sklearn.cluster import DBSCAN 

cluster = DBSCAN(eps=0.7, min_samples=5)
cluster.fit(df_purchase[['ScaledSales', 'ScaledPurchase']])

df_purchase['ClusterDbscan'] = cluster.labels_

df_purchase['ClusterDbscan'] = df_purchase['ClusterDbscan'].map({1:2, -1:1, 0:0, 2:3})
df_purchase



,CustomerID,Sales,ScaledSales,LastPurchase,LastPurchase_days,ScaledPurchase,ClusterDbscan
0,12346.0,0.00,-0.261674,2011-01-18 10:17:00,325,2.337128,0
1,12747.0,4196.01,0.379155,2011-12-07 14:34:00,1,-0.895835,0
2,12748.0,29072.10,4.178320,2011-12-09 12:20:00,0,-0.905813,2
3,12749.0,3868.20,0.329091,2011-12-06 09:56:00,3,-0.875878,0
4,12820.0,942.34,-0.117757,2011-12-06 15:12:00,2,-0.885857,0
...,...,...,...,...,...,...,...
3945,18280.0,180.60,-0.234092,2011-03-07 09:52:00,277,1.858171,0
3946,18281.0,80.82,-0.249331,2011-06-12 10:53:00,180,0.890277,0
3947,18282.0,176.60,-0.234703,2011-12-02 11:43:00,7,-0.835965,0
3948,18283.0,2094.88,0.058263,2011-12-06 12:02:00,3,-0.875878,0


In [611]:
# Clustering by K-means

from sklearn.cluster import KMeans

# elbow method
WCSS=[]
for n in range(1,10):
  cluster = KMeans(n_clusters=n)
  cluster.fit(df_purchase[['ScaledSales', 'ScaledPurchase']])
  WCSS.append(cluster.inertia_)
  
WCSS

fig = px.line(x=range(1,10), y=WCSS, width=500, height=300, template='xgridoff')
fig.show()

In [612]:
cluster = KMeans(n_clusters=4)
cluster.fit(df_purchase[['ScaledSales', 'ScaledPurchase']])
cluster.labels_

df_purchase['ClusterKmeans'] = cluster.labels_
df_purchase.head()

,CustomerID,Sales,ScaledSales,LastPurchase,LastPurchase_days,ScaledPurchase,ClusterDbscan,ClusterKmeans
0,12346.0,0.00,-0.261674,2011-01-18 10:17:00,325,2.337128,0,0
1,12747.0,4196.01,0.379155,2011-12-07 14:34:00,1,-0.895835,0,1
2,12748.0,29072.10,4.178320,2011-12-09 12:20:00,0,-0.905813,2,3
3,12749.0,3868.20,0.329091,2011-12-06 09:56:00,3,-0.875878,0,1
4,12820.0,942.34,-0.117757,2011-12-06 15:12:00,2,-0.885857,0,1


In [613]:
# Clustering by Agglomerative method

from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=4)
cluster.fit(df_purchase[['ScaledSales','ScaledPurchase']])

df_purchase['ClusterAgglomerative'] = cluster.labels_
df_purchase.head()

,CustomerID,Sales,ScaledSales,LastPurchase,LastPurchase_days,ScaledPurchase,ClusterDbscan,ClusterKmeans,ClusterAgglomerative
0,12346.0,0.00,-0.261674,2011-01-18 10:17:00,325,2.337128,0,0,0
1,12747.0,4196.01,0.379155,2011-12-07 14:34:00,1,-0.895835,0,1,1
2,12748.0,29072.10,4.178320,2011-12-09 12:20:00,0,-0.905813,2,3,1
3,12749.0,3868.20,0.329091,2011-12-06 09:56:00,3,-0.875878,0,1,1
4,12820.0,942.34,-0.117757,2011-12-06 15:12:00,2,-0.885857,0,1,1


In [614]:

fig = make_subplots(rows=3 , cols=1, shared_yaxes='all', 
                    column_width=[3], subplot_titles=("DBSCAN", "KMeans", "Agglomerative"))


fig.add_trace(go.Scatter(x=df_purchase['ScaledPurchase'], y= df_purchase["ScaledSales"], mode='markers', marker_color=df_purchase["ClusterDbscan"], showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=df_purchase[df_purchase["ClusterDbscan"]==1]['ScaledPurchase'], 
                           y= df_purchase[df_purchase["ClusterDbscan"]==1]["ScaledSales"],
                           mode='markers', marker_symbol='x', marker_color="lightskyblue", marker_size=10, showlegend=True), row=1, col=1)

fig.add_trace(go.Scatter(x=df_purchase['ScaledPurchase'], y= df_purchase["ScaledSales"], mode='markers', marker_color=df_purchase["ClusterKmeans"], showlegend=True), row=2, col=1)
fig.add_trace(go.Scatter(x=df_purchase['ScaledPurchase'], y= df_purchase["ScaledSales"], mode='markers', marker_color=df_purchase["ClusterAgglomerative"], showlegend=True), row=3, col=1)

fig.update_layout(showlegend=False, template='plotly_dark', width=1000, height=900,)

fig.show()

### BONUS - Classification make_moons and random data


In [615]:
from sklearn.datasets import make_moons

moons_data = make_moons(n_samples=1000, noise=0.05)[0]
df_moons = pd.DataFrame(moons_data, columns=['x1', 'x2'])
fig = px.scatter(df_moons, 'x1', 'x2', width=750, height=400, title='moons data', template='seaborn')
fig.show()

In [616]:
fig = make_subplots(rows=1, cols=3, shared_yaxes=True, horizontal_spacing=0.01)

# KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(df_moons[['x1','x2']])
df_moons['kmeans_cluster'] = kmeans.predict(df_moons[['x1','x2']])


# AgglomerativeClustering
agglo = AgglomerativeClustering(n_clusters=2, affinity='euclidean')
df_moons['agglomerative_cluster'] = agglo.fit_predict(df_moons[['x1','x2']])


# DBSCAN
dbscan = DBSCAN(eps=0.09, min_samples=6)
dbscan.fit(df_moons[['x1','x2']])
df_moons['dbscan_cluster'] = dbscan.labels_


In [617]:
fig = make_subplots(rows=1 , cols=3, subplot_titles=("KMeans", "agglomerative", "DBSCAN"), )


fig.add_trace(px.scatter(df_moons, 'x1', 'x2', 'kmeans_cluster', width=800, height=500)['data'][0], row=1, col=1)
fig.add_trace(px.scatter(df_moons, 'x1', 'x2', 'agglomerative_cluster', width=800, height=500)['data'][0], row=1, col=2)
fig.add_trace(px.scatter(df_moons, 'x1', 'x2', 'dbscan_cluster', width=800, height=500)['data'][0], row=1, col=3)

fig.update_layout( template='plotly_dark', showlegend=False)
fig.show()

In [618]:
random_data = np.random.rand(1000, 2)
df_random = pd.DataFrame(random_data, columns=['x1', 'x2'])
fig = px.scatter(df_random, 'x1', 'x2', width=550, height=550, title='random data', template='seaborn')
fig.show()

In [619]:
# Random data

fig = make_subplots(rows=1, cols=3, shared_yaxes=True, horizontal_spacing=0.01)

# KMeans
kmeans = KMeans(n_clusters=6)
kmeans.fit(df_random[['x1','x2']])
df_random['kmeans_cluster'] = kmeans.predict(df_random[['x1','x2']])


# AgglomerativeClustering
agglo = AgglomerativeClustering(n_clusters=6)
df_random['agglomerative_cluster'] = agglo.fit_predict(df_random[['x1','x2']])

# DBSCAN
dbscan = DBSCAN(eps=0.05, min_samples=6)
dbscan.fit(df_random[['x1','x2']])
df_random['dbscan_cluster'] = dbscan.labels_

df_random.head()

,x1,x2,kmeans_cluster,agglomerative_cluster,dbscan_cluster
0,0.799342,0.952061,3,4,0
1,0.920522,0.725092,3,4,1
2,0.848664,0.852660,3,4,0
3,0.412790,0.212522,2,1,1
4,0.949336,0.353952,1,0,1


In [620]:
fig = make_subplots(rows=1 , cols=3, subplot_titles=("KMeans", "Agglomerative", "DBSCAN"), )


fig.add_trace(px.scatter(df_random, 'x1', 'x2', 'kmeans_cluster', width=800, height=500)['data'][0], row=1, col=1)
fig.add_trace(px.scatter(df_random, 'x1', 'x2', 'agglomerative_cluster', width=800, height=500)['data'][0], row=1, col=2)
fig.add_trace(px.scatter(df_random, 'x1', 'x2', 'dbscan_cluster', width=800, height=500)['data'][0], row=1, col=3)

fig.update_layout( template='plotly_dark', showlegend=False)
fig.show()